In [31]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests 
import datetime
import urllib
import time


edat = 40
hora = "21:30"


cinema = 'http://www.cinemeseixmacia.com/' #Poseu el nom de la pàgina web
navegador = webdriver.Chrome("/Users/eloi/chromedriver") #Obrir un navegador Chrome
#navegador.implicitly_wait(10)
navegador.get(cinema)

hora = datetime.datetime.strptime(hora, '%H:%M')
pelis_seleccionades=[]

elements = navegador.find_elements_by_xpath('//*[@id="rt-mainbody"]/div/div[3]/div')
print("A la cartellera hi ha...:")
for e in elements:
     if e.is_displayed():   
        e = e.find_element_by_class_name("text")
        peli={}
        peli["puntuacio"] = 0
        peli["release"] = ""
        peli["classi"] = e.find_element_by_id("dadespeli").find_element_by_class_name("classi").get_attribute('textContent')
        horaris = e.find_elements_by_class_name("horasessio")
        peli["horaris"] = [datetime.datetime.strptime(h.find_element_by_tag_name("button").get_attribute('textContent').strip(), '%H:%M') for h in horaris]
        peli["nom"] = e.find_element_by_tag_name("a").get_attribute("name").strip()
        if any ([( datetime.timedelta(minutes=0, hours=0)  <= x-hora <= datetime.timedelta( hours=1, minutes=30)) for x in peli["horaris"]]):
            if peli["classi"] == "APTA" or edat > int(peli["classi"][0:2]) :
                peli["trailer"] = e.find_element_by_class_name("peli-boto-trailer")
                pelis_seleccionades.append(peli)
        url_google = 'http://www.google.es/search?q='+ urllib.parse.quote(peli["nom"].lower()+" site:imdb.com") 
        print(url_google)

print("Seleccionades...:")
for peli in pelis_seleccionades:
    
    url = "https://api.themoviedb.org/3/search/movie"
    parameters = {"include_adult":False, "page":1, "region": "ES", "query": peli["nom"],"language":"es-ES", "api_key":"ad38c5a701d46fed6b0ebc7c3d25dd49"  }
    response = requests.get(url,params=parameters)
    if (int(response.headers["x-ratelimit-remaining"])) > 0:
        data = response.json()
        if len(data["results"]) > 0 :
            millor_result = data["results"][0]
            peli["puntuacio"] = millor_result["vote_average"]
            peli["release"] = millor_result["release_date"]
    else:
        print("Torna-ho a intentar")
    print(peli["release"]+" - "+peli["nom"]+ "("+peli["classi"]+") "+ str([ y.time().strftime("%H:%M") for y in peli["horaris"]]) + "*** " + str(peli["puntuacio"]))


ordenada = sorted(pelis_seleccionades, key=lambda k: k['puntuacio'], reverse=True)
ActionChains(navegador).move_to_element(ordenada[0]["trailer"]).perform()
time.sleep(3)
navegador.execute_script("arguments[0].click();", ordenada[0]["trailer"])
time.sleep(3)
wait = WebDriverWait(navegador, 5)
wait.until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, 'iframe')))
try:
    wait = WebDriverWait(navegador, 5)
    wait.until(EC.presence_of_element_located((By.ID, "player")))
    navegador.find_element_by_class_name("ytp-large-play-button ytp-button").click()
except:
    print("Clika!")



print("He acabat!")



A la cartellera hi ha...:
http://www.google.es/search?q=bohemian%20rhapsody%20site%3Aimdb.com
http://www.google.es/search?q=el%20cascanueces%20y%20los%20cuatro%20reinos%20site%3Aimdb.com
http://www.google.es/search?q=el%20fot%C3%B3grafo%20de%20mauthausen%20site%3Aimdb.com
http://www.google.es/search?q=hunter%20killer%20site%3Aimdb.com
http://www.google.es/search?q=infiltrado%20en%20el%20kkklan%20site%3Aimdb.com
http://www.google.es/search?q=la%20noche%20de%20halloween%20site%3Aimdb.com
http://www.google.es/search?q=overlord%20site%3Aimdb.com
http://www.google.es/search?q=pesadilla%202%3A%20noche%20de%20%20halloween%20site%3Aimdb.com
http://www.google.es/search?q=smallfoot%20site%3Aimdb.com
http://www.google.es/search?q=tu%20hijo%20site%3Aimdb.com
http://www.google.es/search?q=venom%20site%3Aimdb.com
http://www.google.es/search?q=vose%20millennium%3A%20lo%20que%20no%20te%20mata%20te%20hace%20m%C3%A1s...%20site%3Aimdb.com
http://www.google.es/search?q=animales%20fant%C3%A1sticos%3A%20los

KeyError: 'x-ratelimit-remaining'